In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models

In [2]:
# Load the dataset from TensorFlow Datasets
dataset_name = 'cats_vs_dogs'
dataset, info = tfds.load(dataset_name, with_info=True, as_supervised=True)

In [3]:
# Split the dataset into train and test sets
train_dataset, test_dataset = dataset['train'].take(10000), dataset['train'].skip(10000).take(2500)

In [4]:
# Define image size and batch size for MobileNetV2
IMG_SIZE = 224  # MobileNetV2 requires images of size 224x224
BATCH_SIZE = 32

# Preprocess images (resize, normalize)
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize to 224x224
    image = preprocess_input(image)  # MobileNetV2 specific preprocessing
    return image, label

In [5]:
# Apply the preprocessing and batching
train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
# Load the pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model (optional: unfreeze later for fine-tuning)
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 6s 1us/step


In [7]:
# Build the top layers for our specific task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Pooling the feature maps from MobileNetV2
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification (cats vs dogs)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
# Train the model
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_acc:.4f}')

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 251ms/step - accuracy: 0.9595 - loss: 0.1037 - val_accuracy: 0.9864 - val_loss: 0.0384
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 78s 250ms/step - accuracy: 0.9866 - loss: 0.0386 - val_accuracy: 0.9880 - val_loss: 0.0349
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 80s 256ms/step - accuracy: 0.9888 - loss: 0.0344 - val_accuracy: 0.9876 - val_loss: 0.0389
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 91s 291ms/step - accuracy: 0.9895 - loss: 0.0296 - val_accuracy: 0.9824 - val_loss: 0.0502
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 288ms/step - accuracy: 0.9919 - loss: 0.0217 - val_accuracy: 0.9896 - val_loss: 0.0361
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 219ms/step - accuracy: 0.9884 - loss: 0.0405
Test Accuracy: 0.9896
